In [ ]:
#apt install python3.12-venv
#python3 -m venv .venv
#source .venv/bin/activate // or select kernel venv in visual studio code
#pip install ipykernel requests aws-requests-auth opensearch-py pandas

In [4]:
# Terraform에서 output값을 가져와서 변수로 지정
import subprocess
import json
import sys
import os

def get_terraform_outputs(output_names):
    outputs = {}
    try:
        # Terraform 상태 파일이 있는 현재 디렉토리에서 명령어 실행 가정
        # '-json' 플래그를 사용하여 출력을 JSON 형식으로 받음
        command = ['terraform', 'output', '-json']
        process = subprocess.run(command, capture_output=True, text=True, check=True, encoding='utf-8')

        # JSON 출력 파싱
        all_outputs = json.loads(process.stdout)

        # 요청된 출력 값만 필터링 (또는 모든 값 반환)
        if not output_names:
            return all_outputs # 모든 출력 반환
        else:
            for name in output_names:
                if name in all_outputs:
                    # 출력 값 구조에서 실제 value 추출
                    outputs[name] = all_outputs[name].get('value', None)
                else:
                    print(f"경고: 출력 값 '{name}'을 찾을 수 없습니다.", file=sys.stderr)
                    outputs[name] = None
            return outputs

    except FileNotFoundError:
        print("오류: 'terraform' 명령어를 찾을 수 없습니다. Terraform이 설치되어 있고 PATH에 있는지 확인하세요.", file=sys.stderr)
        return None
    except subprocess.CalledProcessError as e:
        print(f"오류: 'terraform output' 실행 중 오류 발생 (종료 코드: {e.returncode})", file=sys.stderr)
        print(f"오류 메시지: {e.stderr}", file=sys.stderr)
        return None
    except json.JSONDecodeError as e:
        print(f"오류: Terraform 출력 JSON 파싱 중 오류 발생: {e}", file=sys.stderr)
        print(f"받은 출력: {process.stdout}", file=sys.stderr)
        return None
    except Exception as e:
        print(f"알 수 없는 오류 발생: {e}", file=sys.stderr)
        return None

if __name__ == "__main__":
    # 가져오고 싶은 출력 값 이름 목록
    desired_outputs = ["opensearch_domain_endpoint", "lambda_iam_role_arn"]

    # Terraform 출력 값 가져오기 시도
    retrieved_outputs = get_terraform_outputs(desired_outputs)

    if retrieved_outputs:
        print("\n성공적으로 가져온 Terraform 출력 값:")
        for name, value in retrieved_outputs.items():
            print(f"- {name}: {value}")

        # 예시: 가져온 값을 변수에 할당하여 사용
        opensearch_endpoint = retrieved_outputs.get("opensearch_domain_endpoint")
        lambda_iam_role_arn = retrieved_outputs.get("lambda_iam_role_arn")

        if opensearch_endpoint and lambda_iam_role_arn:
            print("\n스크립트의 다른 부분에서 값 사용 가능:")
            print(f"  OpenSearch Endpoint: {opensearch_endpoint}")
            print(f"  Lambda Role ARN: {lambda_iam_role_arn}")
            # 여기에 이 값들을 사용하는 다음 로직 추가 가능 (예: 역할 매핑 스크립트 호출)
        else:
            print("\n필요한 출력 값 중 일부를 가져오지 못했습니다.")
    else:
        print("\nTerraform 출력 값을 가져오는 데 실패했습니다.")


성공적으로 가져온 Terraform 출력 값:
- opensearch_domain_endpoint: search-integration-log-timangs-pmq42otk4e4kzasqldinbpkgey.ap-northeast-2.es.amazonaws.com
- lambda_iam_role_arn: arn:aws:iam::248189921892:role/lambda-s3-opensearch-role

스크립트의 다른 부분에서 값 사용 가능:
  OpenSearch Endpoint: search-integration-log-timangs-pmq42otk4e4kzasqldinbpkgey.ap-northeast-2.es.amazonaws.com
  Lambda Role ARN: arn:aws:iam::248189921892:role/lambda-s3-opensearch-role


In [5]:
# FGAC Mapping (이거 안하면 Lambda가 권한이 없어버버림)
import requests
import json
import os
import sys
import subprocess # Terraform 명령어 실행을 위해 추가
import base64     # Base64 인코딩/디코딩을 위해 추가
import binascii   # Base64 오류 처리를 위해 추가

def get_terraform_outputs(output_names):

    outputs = {}
    print("Terraform 출력 값 가져오는 중...")
    try:
        command = ['terraform', 'output', '-json']
        process = subprocess.run(command, capture_output=True, text=True, check=True, encoding='utf-8', cwd=os.getcwd()) # 현재 디렉토리에서 실행

        all_outputs = json.loads(process.stdout)

        for name in output_names:
            if name in all_outputs:
                outputs[name] = all_outputs[name].get('value', None)
                if outputs[name] is None:
                     print(f"경고: 출력 값 '{name}'의 'value'를 찾을 수 없습니다.", file=sys.stderr)
            else:
                print(f"경고: 출력 값 '{name}'을 찾을 수 없습니다.", file=sys.stderr)
                outputs[name] = None
        print("Terraform 출력 값 가져오기 완료.")
        return outputs

    except FileNotFoundError:
        print("오류: 'terraform' 명령어를 찾을 수 없습니다. Terraform이 설치되어 있고 PATH에 있는지 확인하세요.", file=sys.stderr)
        return None
    except subprocess.CalledProcessError as e:
        print(f"오류: 'terraform output' 실행 중 오류 발생 (종료 코드: {e.returncode})", file=sys.stderr)
        print(f"오류 메시지: {e.stderr}", file=sys.stderr)
        return None
    except json.JSONDecodeError as e:
        print(f"오류: Terraform 출력 JSON 파싱 중 오류 발생: {e}", file=sys.stderr)
        print(f"받은 출력: {process.stdout}", file=sys.stderr)
        return None
    except Exception as e:
        print(f"알 수 없는 오류 발생 (get_terraform_outputs): {e}", file=sys.stderr)
        return None

# 너무 대놓고 평문이라 최소한의 양심으로 base64 디코딩
def decode_base64_string(encoded_str):
    try:
        # Base64 문자열을 bytes 로 변환 (일반적으로 ASCII 또는 UTF-8)
        encoded_bytes = encoded_str.encode('ascii')
        # Base64 디코딩 수행
        decoded_bytes = base64.b64decode(encoded_bytes)
        # 디코딩된 bytes 를 UTF-8 문자열로 변환
        decoded_string = decoded_bytes.decode('utf-8')
        return decoded_string
    except (binascii.Error, TypeError, ValueError) as e:
        print(f"오류: Base64 디코딩 실패 - {e}", file=sys.stderr)
        return None
    except Exception as e:
        print(f"오류: Base64 디코딩 중 알 수 없는 오류 발생 - {e}", file=sys.stderr)
        return None

TARGET_OPENSEARCH_ROLE = "all_access"
OPENSEARCH_ADMIN_USER = decode_base64_string("YWRtaW4=")
OPENSEARCH_ADMIN_PASSWORD = decode_base64_string("MGwwM2xWJDNAbDJrcmw=")

if __name__ == "__main__":
    print(f"\n사용할 값 확인:")
    print(f"  OpenSearch Endpoint: {opensearch_endpoint}")
    print(f"  Lambda Role ARN: {lambda_iam_role_arn}")
    print(f"  Target OpenSearch Role: {TARGET_OPENSEARCH_ROLE}")
    print(f"  Admin User: {OPENSEARCH_ADMIN_USER}")

    if not OPENSEARCH_ADMIN_PASSWORD:
        print("\n오류: OpenSearch 관리자 비밀번호가 설정되지 않았습니다. OPENSEARCH_ADMIN_PASSWORD 환경 변수를 설정하세요.", file=sys.stderr)
        sys.exit(1)

    api_path = f"_plugins/_security/api/rolesmapping/{TARGET_OPENSEARCH_ROLE}"
    url = f"https://{opensearch_endpoint}/{api_path}"
    headers = {"Content-Type": "application/json"}
    payload = {
        "backend_roles": [
            lambda_iam_role_arn
        ],
        "hosts": [],
        "users": [
            OPENSEARCH_ADMIN_USER
        ]
    }

    print(f"\n'{TARGET_OPENSEARCH_ROLE}' 역할에 IAM 역할 '{lambda_iam_role_arn}' 매핑을 시도합니다...")
    print(f"대상 URL: {url}")

    try:
        response = requests.put(
            url,
            auth=(OPENSEARCH_ADMIN_USER, OPENSEARCH_ADMIN_PASSWORD),
            headers=headers,
            json=payload,
            timeout=30
        )
        response.raise_for_status() 
        print(f"성공! 응답 코드: {response.status_code}")
        print("응답 내용:")
        print(json.dumps(response.json(), indent=2))

    except requests.exceptions.RequestException as e:
        print(f"오류 발생: {e}")
        if e.response is not None:
            print(f"오류 응답 코드: {e.response.status_code}")
            try:
                print(f"오류 응답 내용: {json.dumps(e.response.json(), indent=2)}")
            except json.JSONDecodeError:
                print(f"오류 응답 내용 (Non-JSON): {e.response.text}")
    except Exception as e:
        print(f"알 수 없는 오류 발생: {e}")


사용할 값 확인:
  OpenSearch Endpoint: search-integration-log-timangs-pmq42otk4e4kzasqldinbpkgey.ap-northeast-2.es.amazonaws.com
  Lambda Role ARN: arn:aws:iam::248189921892:role/lambda-s3-opensearch-role
  Target OpenSearch Role: all_access
  Admin User: admin

'all_access' 역할에 IAM 역할 'arn:aws:iam::248189921892:role/lambda-s3-opensearch-role' 매핑을 시도합니다...
대상 URL: https://search-integration-log-timangs-pmq42otk4e4kzasqldinbpkgey.ap-northeast-2.es.amazonaws.com/_plugins/_security/api/rolesmapping/all_access
성공! 응답 코드: 200
응답 내용:
{
  "status": "OK",
  "message": "'all_access' updated."
}


In [6]:
import requests
import json

def create_opensearch_index_pattern(pattern_title, time_field):
    api_path = f"_dashboards/api/saved_objects/_bulk_create"
    url = f"https://{opensearch_endpoint}/{api_path}"
    headers = {
        "Content-Type": "application/json",
        "osd-xsrf": "true"
    }
    payload = [
      {
        "type": "index-pattern",
        "attributes": {
            "title": pattern_title,
            "timeFieldName": time_field
        }
      }
    ]
    print(f"\n인덱스 패턴 '{pattern_title}' 생성을 시도합니다...")
    print(f"대상 URL: {url}")
    try:
        response = requests.post(
            url,
            auth=(OPENSEARCH_ADMIN_USER, OPENSEARCH_ADMIN_PASSWORD),
            headers=headers,
            json=payload,
            timeout=30
        )
        response.raise_for_status() # 2xx 외 상태 코드 시 예외 발생
        print(f"성공! 응답 코드: {response.status_code}")
        # print("처리 결과:")
        response_json = response.json()
        # print(json.dumps(response_json, indent=2))
        return response_json # 성공 시 응답 반환
    except requests.exceptions.RequestException as e:
        print(f"오류 발생: {e}")
        if e.response is not None:
            print(f"오류 응답 코드: {e.response.status_code}")
            try:
                print(f"오류 응답 내용: {json.dumps(e.response.json(), indent=2)}")
            except json.JSONDecodeError:
                print(f"오류 응답 내용 (Non-JSON): {e.response.text}")
                if e.response.status_code == 409:
                     print("참고: 해당 인덱스 패턴이 이미 존재할 수 있습니다.")
        return None
    except Exception as e:
        print(f"알 수 없는 오류 발생: {e}")
        return None

if __name__ == "__main__":
    INDEX_PATTERN_TITLE = ["cloudtrail-*","web-*"]
    TIME_FIELD_NAME = "eventTime"
    for index_pattern in INDEX_PATTERN_TITLE:
        result = create_opensearch_index_pattern(
            pattern_title=index_pattern,
            time_field=TIME_FIELD_NAME
        )
        if result:
            print("\n함수 호출 성공.")
        else:
            print("\n함수 호출 실패.")




인덱스 패턴 'cloudtrail-*' 생성을 시도합니다...
대상 URL: https://search-integration-log-timangs-pmq42otk4e4kzasqldinbpkgey.ap-northeast-2.es.amazonaws.com/_dashboards/api/saved_objects/_bulk_create
성공! 응답 코드: 200

함수 호출 성공.

인덱스 패턴 'web-*' 생성을 시도합니다...
대상 URL: https://search-integration-log-timangs-pmq42otk4e4kzasqldinbpkgey.ap-northeast-2.es.amazonaws.com/_dashboards/api/saved_objects/_bulk_create
성공! 응답 코드: 200

함수 호출 성공.
